In [397]:
#importing all the required librarise

import pandas as pd
from bs4 import BeautifulSoup
import requests
import selenium
import json
import os
from PIL import Image
import io
import hashlib
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import re


1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.


In [3]:
# Connecting to the webbrowser
driver= webdriver.Chrome(r"/Users/prajwalkunder/Downloads/chromedriver") 

In [4]:
# Connecting to the webpage
url="https://www.amazon.in/"
driver.get(url) 

In [5]:
# Taking input from user 
search_string = input("Input the string you want to search for:")

# Generate a url from search string
url= driver.get("https://www.amazon.in/s?k=" + search_string + "&start=")

Input the string you want to search for:laptops


2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
# Creating a function to srape data of all products of first three pages.
urls=[]
for k in range(3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break  

In [7]:
# Creating empty list to store data.
brand_name=[]
name_of_the_product=[]
rating=[]
no_of_ratings=[]
price=[]
expected_delivery=[]
return_exchange=[]
other_details=[]
availability=[]
product_url=[] 

In [8]:
# Creating for loop for urls
for i in urls:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(5) 
    
# Scraping the brand name
    try: brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
    except: brand_name.append('_')
    driver.implicitly_wait(5)
    
#Scraping name of the product
    try:
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+1
        name_of_the_product.append(string)
    except: name_of_the_product.append('_')
    driver.implicitly_wait(5)
    
#Scraping the rating of the product
    try: rating.append(driver.find_element_by_id('acrPopover').get_attribute('title'))
    except: rating.append('_')
    driver.implicitly_wait(5)
    
#Scraping numer of rating of the product
    try: no_of_ratings.append(driver.find_element_by_id('acrCustomerReviewText').text)
    except: no_of_ratings.append('_')
    driver.implicitly_wait(5)
    
#Scraping price of the product
    try:
        try: price.append(driver.find_element_by_id('priceblock_saleprice').text)
        except: price.append(driver.find_element_by_id("priceblock_dealprice").text)
    except: price.append('_')
    driver.implicitly_wait(5)
    
#Scraping the details of return/exchange of the product
    try:
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_element_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                reture_exchange.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            return_exchange.append("_")
    except: return_exchange.append("_")
    driver.implicitly_wait(5)
    
#Scraping expected delivery date
    try: expected_delivery.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: expected_delivery.append('_')
    driver.implicitly_wait(5)
    
#Scraping availability of the product
    try:
        try: availability.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: availability.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except:availability.append("_")
    driver.implicitly_wait(5)
    
#Scraping other details
    try:
        details=[i.text.replace('\n','___')for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")]
        other_details.append(details[0])
    except: other_details.append('_')
    driver.implicitly_wait(5)

In [9]:
#Checking the scraped data lenght
print(len(brand_name),len(name_of_the_product),len(rating),len(no_of_ratings),len(price),len(expected_delivery),len(return_exchange),len(other_details),len(availability),len(product_url))


66 66 66 66 66 66 66 66 66 66


In [10]:
#Creating dataframe
df=pd.DataFrame({}) 

In [12]:
#Creating dataset
df['Brand_name']=brand_name
df['Name_of_the_product']=name_of_the_product
df['Rating']=rating
df['No_of_ratings']=no_of_ratings
df['Price']=price
df['Expected_delivery']=expected_delivery
df['Return_exchange']=return_exchange
df['Other_details']=other_details
df['Availability']=availability
df['Product_url']=product_url


In [13]:
df

,Brand_name,Price,Rating,Name_of_the_product,No_of_ratings,Expected_delivery,Return_exchange,Other_details,Availability,Product_url
0,HP,_,4.0 out of 5 stars,_,1 rating,"Monday, July 26",_,"Brand HP___Manufacturer hp, HP India Sales Pvt...",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,HP,_,_,_,_,"Monday, July 26",_,"Brand HP___Manufacturer HP Inc., HP Inc.___Ser...",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,HP,_,3.4 out of 5 stars,_,549 ratings,"Wednesday, July 28",_,"Brand HP___Manufacturer hp, Manufactured by on...",In stock.,https://www.amazon.in/HP-Pentium-Processor-15-...
3,HP,_,3.6 out of 5 stars,_,"1,372 ratings",_,_,"Brand HP___Manufacturer hp, HP___Model Name 15...",Currently unavailable.,https://www.amazon.in/HP-15-6-inch-Laptop-Wind...
4,ASUS,_,4.0 out of 5 stars,_,767 ratings,July 26 - 28,_,"Brand ASUS___Manufacturer Asus, TECH FRONT ( C...",Available to ship in 1-2 days.,https://www.amazon.in/ASUS-VivoBook-Integrated...
...,...,...,...,...,...,...,...,...,...,...
61,Life,_,2.6 out of 5 stars,_,8 ratings,"Monday, July 26",_,"Brand LifeDigital___Manufacturer LifeDigital, ...",In stock.,https://www.amazon.in/LifeDigital-Laptop-Intel...
62,ASUS,_,4.4 out of 5 stars,_,100 ratings,"Monday, July 26",_,"Brand ASUS___Manufacturer Asus, TECH FRONT ( C...",In stock.,https://www.amazon.in/ASUS-i3-1115G4-15-6-inch...
63,HP,_,4.4 out of 5 stars,_,200 ratings,"Monday, July 26",_,"Brand HP___Manufacturer HP Inc., HP Inc.___Mod...",In stock.,https://www.amazon.in/HP-Pavilion-Graphics-35-...
64,HP,_,4.3 out of 5 stars,_,574 ratings,"Monday, July 26",_,"Brand HP___Manufacturer HP Inc., HP Inc.___Ser...",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...


In [14]:
#Saving file as CSV .
import joblib
joblib.dump(df,"Products.csv") 

['Products.csv']

Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [39]:

# Function Definition
def google_img(url):
    driver=webdriver.Chrome(r"/Users/prajwalkunder/Downloads/chromedriver")
    fruits=[]
    cars=[]
    Bikes=[]
    machine_learning=[]
#extracting fruits data
    driver.get(url)
    search_field_items=driver.find_element_by_xpath("//div[@class='a4bIc']/input")
    search_field_items.send_keys("fruits")
    search_button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    search_button.click()
    for i in range(0,3):
        url_tags=driver.find_elements_by_xpath("//div[@class='islrc']//div[@class='isv-r PNCib MSM1fd BUooTd']//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
        for i in url_tags:
            fruits.append(i.get_attribute('href'))   
        time.sleep(2)
#extracting car image data        
    driver.get(url)
    search_field_items=driver.find_element_by_xpath("//div[@class='a4bIc']/input")
    search_field_items.send_keys("cars")
    search_button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    search_button.click()
    for i in range(0,3):
        url_tags=driver.find_elements_by_xpath("//div[@class='islrc']//div[@class='isv-r PNCib MSM1fd BUooTd']//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
        for i in url_tags:
            cars.append(i.get_attribute('href'))   
        time.sleep(2)
#extracting machine learning imgae data        
    driver.get(url)
    search_field_items=driver.find_element_by_xpath("//div[@class='a4bIc']/input")
    search_field_items.send_keys("machine learning")
    search_button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    search_button.click()
    for i in range(0,3):
        url_tags=driver.find_elements_by_xpath("//div[@class='islrc']//div[@class='isv-r PNCib MSM1fd BUooTd']//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
        for i in url_tags:
            machine_learning.append(i.get_attribute('href'))   
            

#extracting bikes image data        
    driver.get(url)
    search_field_items=driver.find_element_by_xpath("//div[@class='a4bIc']/input")
    search_field_items.send_keys("Bikes")
    search_button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    search_button.click()
    for i in range(0,3):
        url_tags=driver.find_elements_by_xpath("//div[@class='islrc']//div[@class='isv-r PNCib MSM1fd BUooTd']//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
        for i in url_tags:
            Bikes.append(i.get_attribute('href'))   
        
        
    
        
    print('Length of the list are ==>>',len(fruits),len(cars),len(machine_learning),len(Bikes))
    google_image=pd.DataFrame({'Bikes':Bikes[:100],
                           'cars':cars[:100],
                            'machine_learning':machine_learning[:100],
                              'fruits':fruits[:100]})
                            
    print(google_image)
    google_image.to_csv('google_image.csv', index = False)
    
    
# Calling Function
google_img('https://images.google.com/?gws_rd=ssl')

Length of the list are ==>> 144 138 144 144
                                                Bikes  \
0                  https://www.zigwheels.com/newbikes   
1    https://www.bikedekho.com/husqvarna/vitpilen-250   
2   https://www.autocarindia.com/bike-news/best-se...   
3   https://www.livemint.com/auto-news/harley-davi...   
4                  https://www.zigwheels.com/newbikes   
..                                                ...   
95        https://www.designboom.com/tag/bike-design/   
96                 https://www.zigwheels.com/newbikes   
97   https://www.bikedekho.com/husqvarna/vitpilen-250   
98  https://www.autocarindia.com/bike-news/best-se...   
99  https://www.livemint.com/auto-news/harley-davi...   

                                                 cars  \
0   https://www.popularmechanics.com/cars/a3482425...   
1   https://www.forbes.com/sites/jimgorzelany/2019...   
2   https://luxe.digital/lifestyle/cars/fastest-cars/   
3                     https://www.wired.com

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [370]:
#getting url.
url='https://www.flipkart.com/'
driver.get(url) 

In [371]:
#Searching prodect element.
search_bar = driver.find_element_by_class_name("_3704LK")
search_bar.send_keys(' pixel 4A')


In [372]:
#Searching elements.
search_button=driver.find_element_by_class_name('L0Z3Pu')
search_button.click()


In [373]:
#Creating empty list 
brand=[]
displaysize=[]
displayresolution=[]
processor=[]
processorcores=[]
Smartphonename=[]
Colour=[]
RAM=[]
Storage=[]
primarycamera=[]
secondarycamera=[]
batterycapacity=[]
price=[]
producturl=[] 

In [374]:
# Extracting brand name data
brand_name=[]
for i in range(10):
    for i in driver.find_elements_by_xpath("//div[@class='_2gmUFU _3V8rao']"):
        brand_name.append(i.text)
    time.sleep(5)
    
brand_name

['RAM',
 'BRAND',
 'CUSTOMER RATINGS',
 'INTERNAL STORAGE',
 'BATTERY CAPACITY',
 'OPERATING SYSTEM',
 'NETWORK TYPE',
 'SCREEN SIZE',
 'SIM TYPE',
 'PRIMARY CAMERA',
 'SECONDARY CAMERA',
 'OFFERS',
 'PROCESSOR BRAND',
 'SPECIALITY',
 'RESOLUTION TYPE',
 'TYPE',
 'OPERATING SYSTEM VERSION NAME',
 'AVAILABILITY',
 'DISCOUNT',
 'CLOCK SPEED',
 'FEATURES',
 'NUMBER OF CORES',
 'BUDGET',
 'GST INVOICE AVAILABLE',
 'RAM',
 'BRAND',
 'CUSTOMER RATINGS',
 'INTERNAL STORAGE',
 'BATTERY CAPACITY',
 'OPERATING SYSTEM',
 'NETWORK TYPE',
 'SCREEN SIZE',
 'SIM TYPE',
 'PRIMARY CAMERA',
 'SECONDARY CAMERA',
 'OFFERS',
 'PROCESSOR BRAND',
 'SPECIALITY',
 'RESOLUTION TYPE',
 'TYPE',
 'OPERATING SYSTEM VERSION NAME',
 'AVAILABILITY',
 'DISCOUNT',
 'CLOCK SPEED',
 'FEATURES',
 'NUMBER OF CORES',
 'BUDGET',
 'GST INVOICE AVAILABLE',
 'RAM',
 'BRAND',
 'CUSTOMER RATINGS',
 'INTERNAL STORAGE',
 'BATTERY CAPACITY',
 'OPERATING SYSTEM',
 'NETWORK TYPE',
 'SCREEN SIZE',
 'SIM TYPE',
 'PRIMARY CAMERA',
 'SECON

In [375]:
print(len(brand_name))

240


In [376]:
#Extracting displaysize data
displaysize_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//li"):
        displaysize_tags.append(j.text)
        
time.sleep(5)

displaysize_tags

['6 GB RAM | 128 GB ROM',
 '14.76 cm (5.81 inch) Full HD+ Display',
 '12.2MP Rear Camera | 8MP Front Camera',
 '3140 mAh Battery',
 'Qualcomm Snapdragon 730G Processor',
 'OLED Display',
 '1 Year Warranty',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdrag

In [377]:
print(len(displaysize_tags))

1540


In [378]:
#Extracting processor data
processor_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='_3879cV']"):
        processor_tags.append(j.text)
        
time.sleep(5)

In [379]:
processor_tags

['',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',
 'No Cost EMI',
 'Special Price',
 '10% or more',
 '20% or more',
 '30% or more',
 '40% or more',
 '10% and below',
 '',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',
 'No Cost EMI',
 'Special Price',
 '10% or more',
 '20% or more',
 '30% or more',
 '40% or more',
 '10% and below',
 '',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',


In [385]:
print(len(processor_tags))

270


In [381]:
#Extracting processor core data
processorcore_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//li"):
        processorcore_tags.append(j.text)
time.sleep(5)

processorcore_tags

['6 GB RAM | 128 GB ROM',
 '14.76 cm (5.81 inch) Full HD+ Display',
 '12.2MP Rear Camera | 8MP Front Camera',
 '3140 mAh Battery',
 'Qualcomm Snapdragon 730G Processor',
 'OLED Display',
 '1 Year Warranty',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdrag

In [384]:
print(len(processorcore_tags))

1540


In [383]:
#Extracting Smart phone data
smartphone_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='_3879cV']"):
        smartphone_tags.append(j.text)
        
    time.sleep(5)

smartphone_tags 

['',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',
 'No Cost EMI',
 'Special Price',
 '10% or more',
 '20% or more',
 '30% or more',
 '40% or more',
 '10% and below',
 '',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',
 'No Cost EMI',
 'Special Price',
 '10% or more',
 '20% or more',
 '30% or more',
 '40% or more',
 '10% and below',
 '',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',


In [386]:
print(len(smartphone_tags))

270


In [57]:
# Extracting color data
color_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//li"):
        color_tags.append(j.text)
    time.sleep(5)
color_tags

['6 GB RAM | 128 GB ROM',
 '14.76 cm (5.81 inch) Full HD+ Display',
 '12.2MP Rear Camera | 8MP Front Camera',
 '3140 mAh Battery',
 'Qualcomm Snapdragon 730G Processor',
 'OLED Display',
 '1 Year Warranty',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdrag

In [387]:
print(len(color_tags))

1530


In [59]:
#Extracting ram data
ram_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//li"):
        ram_tags.append(j.text)
        
time.sleep(5)

ram_tags

['6 GB RAM | 128 GB ROM',
 '14.76 cm (5.81 inch) Full HD+ Display',
 '12.2MP Rear Camera | 8MP Front Camera',
 '3140 mAh Battery',
 'Qualcomm Snapdragon 730G Processor',
 'OLED Display',
 '1 Year Warranty',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdrag

In [388]:
print(len(ram_tags))

1530


In [60]:
#Extracting Storage data
storage_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//li"):
        storage_tags.append(j.text)
    time.sleep(5)

storage_tags

['6 GB RAM | 128 GB ROM',
 '14.76 cm (5.81 inch) Full HD+ Display',
 '12.2MP Rear Camera | 8MP Front Camera',
 '3140 mAh Battery',
 'Qualcomm Snapdragon 730G Processor',
 'OLED Display',
 '1 Year Warranty',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdrag

In [389]:
print(len(storage_tags))

1530


In [62]:
#Extracting primary camara data
primarycamera_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//li"):
        primarycamera_tags.append(j.text)
        
    time.sleep(5)
    
primarycamera_tags

['6 GB RAM | 128 GB ROM',
 '14.76 cm (5.81 inch) Full HD+ Display',
 '12.2MP Rear Camera | 8MP Front Camera',
 '3140 mAh Battery',
 'Qualcomm Snapdragon 730G Processor',
 'OLED Display',
 '1 Year Warranty',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdragon 662 Processor',
 '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase',
 '4 GB RAM | 64 GB ROM',
 '16.59 cm (6.53 inch) Full HD+ Display',
 '48MP + 8MP + 2MP + 2MP | 8MP Front Camera',
 '6000 mAh Battery',
 'Qualcomm Snapdrag

In [79]:
print(len(primarycamera_tags))

1530


In [64]:
#Extracting battery capacity data
batterycapacity_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//div[@class='_3879cV']"):
        batterycapacity_tags.append(j.text)
    time.sleep(5)

batterycapacity_tags

['',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',
 'No Cost EMI',
 'Special Price',
 '10% or more',
 '20% or more',
 '30% or more',
 '40% or more',
 '10% and below',
 '',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',
 'No Cost EMI',
 'Special Price',
 '10% or more',
 '20% or more',
 '30% or more',
 '40% or more',
 '10% and below',
 '',
 '6 GB & Above',
 '4 GB',
 '3 GB',
 '2 GB',
 '1 GB',
 '512 MB - 1 GB',
 'Less than 512 MB',
 'SAMSUNG',
 'Mi',
 'OPPO',
 'LG',
 'Nokia',
 'Google',
 '4★ & above',
 '3★ & above',
 '2★ & above',
 '1★ & above',
 'Buy More, Save More',
 'Exchange Offer',


In [80]:
print(len(batterycapacity_tags))

270


In [66]:
#Extracting the price data
price_tags=[]

for i in range(10):
    for j in driver.find_elements_by_xpath("//span"):
        price_tags.append(j.text)
    time.sleep(5)
    
price_tags

['Plus',
 'Cart',
 'Electronics',
 'TVs & Appliances',
 'Men',
 'Women',
 'Baby & Kids',
 'Home & Furniture',
 'Sports, Books & More',
 'Filters',
 'CATEGORIES',
 '',
 '',
 'PRICE',
 '?',
 '47 MORE',
 'Need help?',
 'Help me decide',
 'Showing 1 – 24 of 308 results for " pixel 4A"',
 'pixel 4A',
 'Sort By',
 '',
 'Add to Compare',
 '4.5',
 '26,261 Ratings & 4,447 Reviews',
 '26,261 Ratings & 4,447 Reviews',
 '26,261 Ratings ',
 '&',
 ' 4,447 Reviews',
 '',
 'Add to Compare',
 '4.3',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings ',
 '&',
 ' 7,194 Reviews',
 '21% off',
 '',
 'Add to Compare',
 '4.3',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings ',
 '&',
 ' 7,194 Reviews',
 '21% off',
 '',
 'Add to Compare',
 '4.3',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings ',
 '&',
 ' 7,194 Reviews',
 '21% off',
 '',
 'Add to Compare',
 '4.3',
 '21,639 Ratings 

In [81]:
print(len(price_tags))

3040


In [68]:
#Extracting rating data.
rating_tags=[]
for i in range(10):
    for j in driver.find_elements_by_xpath("//span"):
        rating_tags.append(j.text)
    time.sleep(5)

rating_tags

['Plus',
 'Cart',
 'Electronics',
 'TVs & Appliances',
 'Men',
 'Women',
 'Baby & Kids',
 'Home & Furniture',
 'Sports, Books & More',
 'Filters',
 'CATEGORIES',
 '',
 '',
 'PRICE',
 '?',
 '47 MORE',
 'Need help?',
 'Help me decide',
 'Showing 1 – 24 of 308 results for " pixel 4A"',
 'pixel 4A',
 'Sort By',
 '',
 'Add to Compare',
 '4.5',
 '26,261 Ratings & 4,447 Reviews',
 '26,261 Ratings & 4,447 Reviews',
 '26,261 Ratings ',
 '&',
 ' 4,447 Reviews',
 '',
 'Add to Compare',
 '4.3',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings ',
 '&',
 ' 7,194 Reviews',
 '21% off',
 '',
 'Add to Compare',
 '4.3',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings ',
 '&',
 ' 7,194 Reviews',
 '21% off',
 '',
 'Add to Compare',
 '4.3',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings & 7,194 Reviews',
 '1,02,329 Ratings ',
 '&',
 ' 7,194 Reviews',
 '21% off',
 '',
 'Add to Compare',
 '4.3',
 '21,639 Ratings 

In [82]:
print(len(rating_tags))

3040


In [366]:
df= pd.DataFrame({}) 

In [369]:
#Creating dataset

df['Brand name']=brand_name[:240]
df['Display size']=displaysize_tags[:240]
df['Processor']=processor_tags[:240]
df['Processor cores']=processorcore_tags[:240]
df['Smart phone name']=smartphone_tags[:240]
df['Colour']=color_tags[:240]
df['Ram']=ram_tags[:240]
df['Storage']=storage_tags[:240]
df['Primary camera']=primarycamera_tags[:240]
df['Battery capacity']=batterycapacity_tags[:240]
df['Price']=price_tags[:240]
df['Rating']=rating_tags[:240]


In [86]:
df

,Brand name,Display size,Display resolution,Processor,Processor cores,Smart phone name,Colour,Ram,Storage,Primary camera,Battery capacity
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 GB & Above
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 GB
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 GB
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 GB
...,...,...,...,...,...,...,...,...,...,...,...
265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10% or more
266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20% or more
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30% or more
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40% or more


In [390]:
df.head()

,Brand name,Display size,Processor,Processor cores,Smart phone name,Colour,Ram,Storage,Primary camera,Battery capacity,Price,Rating
0,RAM,6 GB RAM | 128 GB ROM,,6 GB RAM | 128 GB ROM,,6 GB RAM | 128 GB ROM,6 GB RAM | 128 GB ROM,6 GB RAM | 128 GB ROM,6 GB RAM | 128 GB ROM,,Plus,Plus
1,BRAND,14.76 cm (5.81 inch) Full HD+ Display,6 GB & Above,14.76 cm (5.81 inch) Full HD+ Display,6 GB & Above,14.76 cm (5.81 inch) Full HD+ Display,14.76 cm (5.81 inch) Full HD+ Display,14.76 cm (5.81 inch) Full HD+ Display,14.76 cm (5.81 inch) Full HD+ Display,6 GB & Above,Cart,Cart
2,CUSTOMER RATINGS,12.2MP Rear Camera | 8MP Front Camera,4 GB,12.2MP Rear Camera | 8MP Front Camera,4 GB,12.2MP Rear Camera | 8MP Front Camera,12.2MP Rear Camera | 8MP Front Camera,12.2MP Rear Camera | 8MP Front Camera,12.2MP Rear Camera | 8MP Front Camera,4 GB,Electronics,Electronics
3,INTERNAL STORAGE,3140 mAh Battery,3 GB,3140 mAh Battery,3 GB,3140 mAh Battery,3140 mAh Battery,3140 mAh Battery,3140 mAh Battery,3 GB,TVs & Appliances,TVs & Appliances
4,BATTERY CAPACITY,Qualcomm Snapdragon 730G Processor,2 GB,Qualcomm Snapdragon 730G Processor,2 GB,Qualcomm Snapdragon 730G Processor,Qualcomm Snapdragon 730G Processor,Qualcomm Snapdragon 730G Processor,Qualcomm Snapdragon 730G Processor,2 GB,Men,Men


In [391]:
df.to_csv('smartphone.csv')


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [43]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                       
search = driver.find_element_by_id("searchboxinput")                    
search.clear()                                                             
time.sleep(2)
search.send_keys(city)                                                    
button = driver.find_element_by_id("searchbox-searchbutton")              
button.click()                                                          
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e)) 

Enter City Name : jaipur
URL Extracted:  https://www.google.co.in/maps/place/Jaipur,+Rajasthan/@15.0172605,76.3178081,7z/data=!4m5!3m4!1s0x396c4adf4c57e281:0xce1c63a0cf22e09!8m2!3d26.9124336!4d75.7872709
Latitude = 15.0172605, Longitude = 76.3178081


6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [46]:

url = 'https://trak.in'
driver.get(url)


In [47]:
#Selecting Search Button.
search_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(search_button)

In [48]:

#creating Empty List:
trade_dict = {}
trade_dict['Date'] = []
trade_dict['Startup Name'] = []
trade_dict['Industry/Vertical'] = []
trade_dict['Sub-Vertical'] = []
trade_dict['Location'] = []
trade_dict['Investor'] = []
trade_dict['Investment Type'] = []
trade_dict['Amount(in USD)'] = [] 

In [49]:
for i in range(40,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

# Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        trade_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        trade_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        trade_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        trade_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        trade_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        trade_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        trade_dict['Investment Type'].append(n.text)
        
     # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        trade_dict['Amount(in USD)'].append(a.text)
    
trade_df = pd.DataFrame(trade_dict)
trade_df.head() 

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,13/11/2019,Freshworks,Software,Business and customer engagement tools,San Francisco,"Sequoia, CapitalG, Accel",Series H,"15,000,0000"
1,14/11/2019,Misters,Health and wellness,Men's Health and Wellness brand,Gurgaon,"Sauce.vc, Rainforest Ventures",Series B,"486,000"
2,13/11/2019,Sunstone Eduversity Pvt. Ltd,Education,Elearning,Gurgaon,"Prime Venture Partners, LetsVenture, PS1 Ventu...",Seed,"1,500,000"
3,17/11/2019,Burger Singh,Food and Beverage,Indian Burger Brand,Gurgaon,RB Investments,Venture,undisclosed
4,18/11/2019,Healthians,Health and Wellness,Healthcare services,Gurgaon,"DG Daiwa Ventures, DG Incubation",Series B,"12,000,000"


7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [300]:
#connecting to the webdriver
driver = webdriver.Chrome(r"/Users/prajwalkunder/Downloads/chromedriver")

In [301]:
#specifying the url of the webpage to be scraped
url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [302]:
#opening the url through our driver
driver.get(url)

In [303]:
#finding all the seller tags
seller = driver.find_elements_by_class_name("smmerchant")
seller[0:10] 

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="5941d1af-3a27-4069-92de-d2745ac0ba9d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="79b4c2be-9127-450f-a675-85411ea4c7c4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="d212efac-0c4f-4abc-9e64-1780e526fda2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="82be21e4-f3b6-4ab7-8bd6-f8b29be91be4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="c912d2cf-151b-4059-b36b-e00a6bd82c4f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="971cffb5-f587-48b4-9468-248199d593a7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="07c173eb-aedf-47aa-b507-bf

In [304]:
#scraping Seller from seller tags
Seller=[]
for i in seller:
    sellers=i.text.replace("\n"," ")
    Seller.append(sellers)
Seller 

['N/A', 'N/A', 'N/A', 'amazon', 'amazon', 'amazon', 'amazon']

In [305]:
#finding all the price tags
price = driver.find_elements_by_class_name("smprice")
price[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="7c32aa55-12ff-46c4-8ed5-10d2f25f7eb6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="324ad69c-2590-4b6d-bc4d-cbd72ce19637")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="a749e766-da85-4ee6-b169-dab8c2555127")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="438b7e10-4bf9-49c4-b899-2dd53e26fd0c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="2e439001-35e7-4483-8880-50245ab4dc73")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="d290861b-5dfa-4eca-b8e6-25b61037e00a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="0338a829-1595-47dc-a7ee-98

In [306]:
#scraping price from price tags
PRICE=[]
for i in price:
    prices=i.text.replace("\n"," ")
    PRICE.append(prices)
PRICE

['N/A', '₹341990', 'N/A', '₹164990', '₹76988', '₹185000', '₹79400']

In [307]:
#finding all the os tags
os = driver.find_elements_by_xpath("//*[contains(text(), 'OS')]")
os[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="a1f1dc57-b742-4924-ac7a-9520c1f8693b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="8d347d91-daae-4da5-bf84-f60a9cf84988")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="2126c822-ed58-46c8-b310-78b68cf40e43")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="092277c7-f22e-49ca-9d8b-8c71c9f6838c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="2b05947e-14f9-4f5e-8411-0873994f6e67")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="534e6752-f689-4af7-8d86-a04ae172ad30")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="d36c9925-acee-467c-b23f-32

In [308]:
#scraping os from tags
OS=[]
for i in os:
    os_=i.text.replace("\n"," ")
    OS.append(os_)
OS

['',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS',
 'OS WINDOWS 10 PRO',
 'OS',
 'OS WINDOWS 10',
 'OS',
 'OS WINDOWS 10 HOME',
 'OS']

In [358]:
#slicing only odd elements from list
OS_ = OS[1::2]
OS_

['OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 HOME',
 'OS WINDOWS 10 PRO',
 'OS WINDOWS 10',
 'OS WINDOWS 10 HOME']

In [310]:
Operating_system = list(map(lambda x: x.replace('OS ',''),OS_))
Operating_system

['WINDOWS 10 HOME',
 'OS',
 'WINDOWS 10 HOME',
 'OS',
 'WINDOWS 10 HOME',
 'OS',
 'WINDOWS 10 HOME',
 'OS',
 'WINDOWS 10 PRO',
 'OS',
 'WINDOWS 10',
 'OS',
 'WINDOWS 10 HOME',
 'OS']

In [311]:
#finding all the display tags
display = driver.find_elements_by_xpath("//*[contains(text(), 'Display')]")
display[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="cabe6891-35d5-4d60-8ff8-1b18e97ee43b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="38747ae5-41d8-41fa-84c2-79e8443a3d46")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="5ef028a7-6165-4884-9ecd-803d3c452103")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="2977277f-fb6e-48bf-98a3-9ba7a09f03fc")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="8b1b4bdc-14f6-4d7e-9de1-b69e63e9bdb9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="5afc1f5c-9cd0-4a5e-9e65-62100ed15887")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="15a87783-1468-49d5-8846-3e

In [312]:
#scraping os from tags
Display=[]
for i in display:
    display_=i.text.replace("\n"," ")
    Display.append(display_)
Display

['DISPLAY 17.3" (1920 X 1080)',
 'Display',
 'DISPLAY 15.6" (3840 X 2160)',
 'Display',
 'DISPLAY 15.6" (1920 X 1080)',
 'Display',
 'DISPLAY 14" (1920 X 1080)',
 'Display',
 'DISPLAY 15.6" (1920 X 1080)',
 'Display',
 'DISPLAY 15.6" (3840 X 1100)',
 'Display',
 'DISPLAY 15.6" (1920 X 1080)',
 'Display',
 '']

In [313]:
#poping unwanted elements
indices = 1,3,5,7,9,11,12,14,16,18,20,21
Display = [i for j, i in enumerate(Display) if j not in indices]
Display

['DISPLAY 17.3" (1920 X 1080)',
 'DISPLAY 15.6" (3840 X 2160)',
 'DISPLAY 15.6" (1920 X 1080)',
 'DISPLAY 14" (1920 X 1080)',
 'DISPLAY 15.6" (1920 X 1080)',
 'DISPLAY 15.6" (3840 X 1100)',
 'Display']

In [314]:
Display = list(map(lambda x: x.replace('DISPLAY ',''),Display))
Display

['17.3" (1920 X 1080)',
 '15.6" (3840 X 2160)',
 '15.6" (1920 X 1080)',
 '14" (1920 X 1080)',
 '15.6" (1920 X 1080)',
 '15.6" (3840 X 1100)',
 'Display']

In [315]:
#finding all the processor tags
processor = driver.find_elements_by_xpath("//*[contains(text(), 'Processor')]")
processor[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="7577238e-6ae3-4269-ad5f-cf33d9b78c8c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="f891a02c-07b0-4d45-ac01-a0d12ea0b22e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="31a160d7-e526-4248-b00a-37185a9df810")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="2a41074b-bb60-4894-8561-a1182ff4294c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="56485698-3d03-49f9-9f44-525ba9104e96")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="9aace86a-29bf-4c33-b2c2-e395cbdb29ad")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="acd52e9b-0bb9-43d9-9187-c5

In [316]:
#scraping processor from tags
Processor=[]
for i in processor:
    processor_=i.text.replace("\n"," ")
    Processor.append(processor_)
Processor

['PROCESSOR 10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR 10TH GENERATION INTEL® CORE™ I9-10980HK | NA',
 'Processor',
 'Graphics Processor',
 'PROCESSOR AMD RYZEN™ 9 5900HX | 3.3 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR 10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ',
 'Processor',
 'Graphics Processor',
 'PROCESSOR INTEL CORE I7 10TH GEN 10875H | NA',
 'Processor',
 'Graphics Processor',
 'PROCESSOR AMD RYZEN™ 5-5500U HEXA-CORE | NA',
 'Processor',
 'Graphics Processor',
 'Best Laptops with Intel Core i5 Processor']

In [317]:
#keeping required elements
indices=[0,3,6,9,11,13,15]
Processor = [ Processor[index] for index in indices ]
Processor 

['PROCESSOR 10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ',
 'PROCESSOR 10TH GENERATION INTEL® CORE™ I9-10980HK | NA',
 'PROCESSOR AMD RYZEN™ 9 5900HX | 3.3 GHZ',
 'PROCESSOR AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ',
 'Graphics Processor',
 'Processor',
 'PROCESSOR INTEL CORE I7 10TH GEN 10875H | NA']

In [318]:
Processor = list(map(lambda x: x.replace('PROCESSOR ',''),Processor))
Processor

['10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ',
 '10TH GENERATION INTEL® CORE™ I9-10980HK | NA',
 'AMD RYZEN™ 9 5900HX | 3.3 GHZ',
 'AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ',
 'Graphics Processor',
 'Processor',
 'INTEL CORE I7 10TH GEN 10875H | NA']

In [319]:
#finding all the Memory tags
memory = driver.find_elements_by_xpath("//*[contains(text(), 'Memory')]")
memory[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="077350c6-cc05-46cf-a882-9782377a973a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="c1b8ee5a-111f-42d1-81bb-1e1add0afa89")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="e935c787-6330-4537-a3fc-6578d64f07b4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="bba58004-210f-48db-ba8e-85f6329d7776")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="37847d6e-7b51-4a2a-9ee7-57b176df5588")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="72aed623-d5ef-42cf-a416-36a68cf65c8d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="5991df21-80b4-4b3e-b328-a6

In [320]:
#scraping Memory from tags
Memory=[]
for i in memory:
    memory_=i.text.replace("\n"," ")
    Memory.append(memory_)
Memory

['MEMORY 1 TB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'Memory',
 'MEMORY 512 GB SSD/4 GBGB DDR4',
 'Memory',
 'MEMORY 512 GB SSD/8 GBGB DDR4',
 'Memory']

In [321]:
#keeping required elements
indices=[0,2,4,6,8,10,12]
Memory = [ Memory[index] for index in indices ]
Memory

['MEMORY 1 TB SSD/16 GBGB DDR4',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'MEMORY 1 TB SSD/16 GBGB DDR4',
 'MEMORY 512 GB SSD/4 GBGB DDR4',
 'MEMORY 512 GB SSD/8 GBGB DDR4']

In [322]:
Memory = list(map(lambda x: x.replace('MEMORY ',''),Memory))
Memory

['1 TB SSD/16 GBGB DDR4',
 '1 TB SSD/16 GBGB DDR4',
 '1 TB SSD/16 GBGB DDR4',
 '1 TB SSD/16 GBGB DDR4',
 '1 TB SSD/16 GBGB DDR4',
 '512 GB SSD/4 GBGB DDR4',
 '512 GB SSD/8 GBGB DDR4']

In [323]:
#finding all the Weight tags
weight = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
weight[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="3457f8bc-e378-49cd-a8bf-b1dca59228d6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="fd349601-a6dc-44a6-8140-627a7a8931d5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="83548b68-f9be-4cd6-b886-2176064814c1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="eb223a0c-74a1-4c0e-958f-15f03f8f2543")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="9f24b223-87d9-4896-984c-e6eaebe315ad")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="1d0c9926-a89b-454d-9131-c6e656f7fa77")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="aa9d2fc8-3e9e-4caa-9d4d-fb

In [324]:
#scraping Weight from tags
Weight=[]
for i in weight:
    weight_=i.text.replace("\n"," ")
    Weight.append(weight_)
Weight

['4.1', 'NA', '2.30', '1.65', '2.3', '2.4', '2.15']

In [325]:
#finding all the dimension tags
graphics = driver.find_elements_by_xpath("//*[contains(text(), 'TX')]")
graphics[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="073c914e-28ea-4178-a374-fcd118ad13d1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="84951738-f605-4f7f-ad72-0385affc44b9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="ed6f6008-58c8-49ec-bfc8-d0f1b1b49a07")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="daef5757-9a97-45d1-a2b1-317228e8a9f6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="0a6c50cc-2dea-474f-9153-6592961c89fb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="a2b1c5e2-f1f1-4a95-ad04-ce04c84bc22a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="981e1b97-8e08-48df-9a6f-12

In [326]:
#scraping dimension from tags
Graphics=[]
for i in graphics:
    graphics_=i.text.replace("\n"," ")
    Graphics.append(graphics_)
Graphics

['',
 "If you want the absolute best when it comes to performance, then the Alienware Area 51M should be on top of your list. It is essentially a desktop replacement that leaves no stones unturned when it comes to delivering pure performance for gaming. It's basically a desktop in disguise, featuring up to 10th gen Intel Core i9K processors and Nvidia RTX 20-series GPU. It's worth pointing out that Area 51M is not exactly suitable for carrying around often. It's a desktop replacement that can be moved around but it's best to set it on a desk. All the variants feature a 71.1-inch display and you can configure it with panels that offer up to 4K resolution.",
 "When the ASUS ROG Strix Scar 15 ended up on our test bench, we immediately knew it was overkill. But you know what? It's a good kind of overkill. ASUS' annual upgrade to the Scar lineup saw some significant changes this year and we like most of them. The Asus ROG Strix Scar 15 comes with up to AMD Ryzen 9 5900HX and Nvidia GeForce 

In [327]:
#keeping required elements
indices=[0,2,4,6,8,10,12]
Graphics = [ Graphics[index] for index in indices ]
Graphics

['',
 "When the ASUS ROG Strix Scar 15 ended up on our test bench, we immediately knew it was overkill. But you know what? It's a good kind of overkill. ASUS' annual upgrade to the Scar lineup saw some significant changes this year and we like most of them. The Asus ROG Strix Scar 15 comes with up to AMD Ryzen 9 5900HX and Nvidia GeForce RTX 3080. The laptop also features per-key RGB on its opto-mechanical keyboard. The new Scar laptops also have a slightly new design, making them look a little different from the older ones. It's also worth pointing out that the laptop comes with a 15.6-inch 1440p display with a refresh rate of 165Hz.",
 'NVIDIA® GeForce RTX™ 3070',
 'NVIDIA GeForce RTX 2060',
 'NVIDIA® GeForce® GTX 1650 4GB',
 'NVIDIA GeForce RTX 2070 Max-Q',
 'NVIDIA® GeForce® GTX 1650']

In [328]:
#finding all the description tags
description = driver.find_elements_by_xpath("//div[@class='Section-center']/p")
description[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="84951738-f605-4f7f-ad72-0385affc44b9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="d8815c06-d150-42f7-b419-e13d0ffe0d0f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="ed6f6008-58c8-49ec-bfc8-d0f1b1b49a07")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="a2b1c5e2-f1f1-4a95-ad04-ce04c84bc22a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="8f648aaf-b703-444a-8641-17bf2e60f588")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="bf7c1023-25cb-4b3e-ae3d-b5a634f31b80")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="1606b9bc-7247-475c-9ad6-32

In [329]:
#scraping description from tags
Description=[]
for i in description:
    description_=i.text.replace("\n"," ")
    Description.append(description_)
Description

["If you want the absolute best when it comes to performance, then the Alienware Area 51M should be on top of your list. It is essentially a desktop replacement that leaves no stones unturned when it comes to delivering pure performance for gaming. It's basically a desktop in disguise, featuring up to 10th gen Intel Core i9K processors and Nvidia RTX 20-series GPU. It's worth pointing out that Area 51M is not exactly suitable for carrying around often. It's a desktop replacement that can be moved around but it's best to set it on a desk. All the variants feature a 71.1-inch display and you can configure it with panels that offer up to 4K resolution.",
 "If the Area 51M is a bit too much for you then you might want to check out the Alienware m15 R3. It's a more portable offering from the folks over at Alienware but it still packs the punch to deliver solid performance. The company hasn't updated these laptops in India with the latest 11th gen Intel Core processors, but a refresh is due 

In [330]:
#finding all the laptop_name tags
laptop = driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
laptop[0:10]

[<selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="ac5fb910-8d6a-4167-96d7-1d6ec9bcaafe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="18f68543-639b-4a18-abd3-339c7dc14bac")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="a55bc10b-9047-47c3-946d-a6ad90ad9f57")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="25a24a6b-2e46-44ef-b881-865baeddf401")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="b77c5e37-1cca-46da-8e50-6c78fffb9732")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="8f63eb77-840c-491d-9b96-70e6661fec68")>,
 <selenium.webdriver.remote.webelement.WebElement (session="65e8b6e90a568ccbca1e2be9195f01ea", element="74ab58fe-8638-4021-80ed-36

In [331]:
#scraping laptop_name from tags
Laptop_name=[]
for i in laptop:
    laptop_=i.text.replace("\n"," ")
    Laptop_name.append(laptop_)
Laptop_name

['ALIENWARE AREA 51M R2',
 'ALIENWARE M15 R3',
 'ASUS ROG STRIX SCAR 15',
 'ASUS ROG ZEPHYRUS G14',
 'LENOVO LEGION 5I',
 'ASUS ROG ZEPHYRUS DUO 15',
 'ACER ASPIRE 7 GAMING']

In [332]:
#getting the list of url of laptops
Laptop_URL=[]

laptop=driver.find_elements_by_xpath("//div[@class='TopNumbeHeading active sticky-footer']")
for i in laptop:
    laptop1=i.find_element_by_tag_name("a")
    Laptop_URL.append(laptop1.get_attribute('href'))

In [333]:
Laptop_URL

['https://www.digit.in/laptops/alienware-area-51m-r2-price-228193.html',
 'https://www.digit.in/laptops/alienware-m15-r3-price-228207.html',
 'https://www.digit.in/laptops/asus-rog-strix-scar-15-price-228221.html',
 'https://www.digit.in/laptops/asus-rog-zephyrus-g14-price-209173.html',
 'https://www.digit.in/laptops/lenovo-legion-5i-price-228235.html',
 'https://www.digit.in/laptops/asus-rog-zephyrus-duo-15-price-215799.html',
 'https://www.digit.in/laptops/acer-aspire-7-gaming-price-223781.html']

In [334]:
#going to the url of first laptop through our driver
driver.get(Laptop_URL[0])

In [335]:
#creating empty lists
Market_status=[]
Release_date=[]
Official_website=[]
Display_rating=[]
CPU_rating=[]
GPU_rating=[]
RAM_rating=[]
Storage_rating=[]
Overall_rating=[]
Availibility=[]
Digit_rating=[]
User_rating=[]
Pros=[]
Cons=[]

In [336]:
#running a loop to extract all the required information from the Laptop_URL

for i in (Laptop_URL):
    driver.get(i)
    
    try:
        market=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.status-bar.only-desktop > div:nth-child(1) > b').text
        Market_status.append(market)
    except NoSuchElementException:
        Market_status.append("-")
    try:
        release=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.status-bar.only-desktop > div:nth-child(2) > b').text
        Release_date.append(release)
    except NoSuchElementException:
        Release_date.append("-")
        
    try:
        website=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.status-bar.only-desktop > div:nth-child(3) > b').text
        Official_website.append(website)
    except NoSuchElementException:
        Official_website.append("-")
        
    try:
        display=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(2) > div > span').text
        Display_rating.append(display)
    except NoSuchElementException:
        Display_rating.append("-")
        
    try:
        cpu=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(3) > div > span').text
        CPU_rating.append(cpu)
    except NoSuchElementException:
        CPU_rating.append("-")
        
    try:
        gpu=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(4) > div > span').text
        GPU_rating.append(gpu)
    except NoSuchElementException:
        GPU_rating.append("-")
        
    try:
        ram=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(5) > div > span').text
        RAM_rating.append(ram)
    except NoSuchElementException:
        RAM_rating.append("-")
        
    try:
        storage=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(6) > div > span').text
        Storage_rating.append(storage)
    except NoSuchElementException:
        Storage_rating.append("-")
        
    try:
        overall=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.Key-specs > div:nth-child(7) > div > span').text
        Overall_rating.append(overall)
    except NoSuchElementException:
        Overall_rating.append("-")
        
        
    try:
        availibility=driver.find_element_by_css_selector('#price > div > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text
        Availibility.append(availibility)
    except NoSuchElementException:
        Availibility.append("-")
        
    try:
        digit=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.score-Bar > ul > li:nth-child(1) > p').text
        Digit_rating.append(digit)
    except NoSuchElementException:
        Digit_rating.append("-")
        
    try:
        user=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.moz-fixed > div > div.col-sm-8.only-desktop > div.score-Bar > ul > li:nth-child(2) > p').text
        User_rating.append(user)
    except NoSuchElementException:
        User_rating.append("-")
        
    try:
        pros=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.wrap-rat > div > div.col-md-4 > div > ul:nth-child(1)').text.replace("\n", " ")
        Pros.append(pros)
    except NoSuchElementException:
        Pros.append("-")
        
    try:
        cons=driver.find_element_by_css_selector('body > div.Center-block > div > div > div.wrap-rat > div > div.col-md-4 > div > ul:nth-child(2)').text.replace("\n", " ")
        Cons.append(cons)
    except NoSuchElementException:
        Cons.append("-")
        
    time.sleep(10)

In [337]:
Market_status

['LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED',
 'LAUNCHED']

In [338]:
Release_date

['28 Apr, 2021',
 '28 Apr, 2021',
 '28 Apr, 2021',
 '12 Aug, 2020',
 '28 Apr, 2021',
 '20 Nov, 2020',
 '09 Apr, 2021']

In [339]:
Official_website

['Alienware', 'Alienware', 'Asus', 'Asus', 'Lenovo', 'Asus', 'Acer']

In [340]:
Display_rating

['-', '-', '-', '-', '-', '-', '-']

In [341]:
CPU_rating

['-', '-', '-', '-', '-', '-', '-']

In [342]:
GPU_rating

['-', '-', '-', '-', '-', '-', '-']

In [343]:
RAM_rating

['-', '-', '-', '-', '-', '-', '-']

In [344]:
Storage_rating

['-', '-', '-', '-', '-', '-', '-']

In [345]:
Overall_rating

['-', '-', '-', '-', '-', '-', '-']

In [346]:
Availibility

['-', '-', '-', 'Available', 'Available', 'Available', 'Available']

In [347]:
Digit_rating

['-', '-', '73', '71', '-', '-', '75']

In [348]:
User_rating

['-', '-', '-', '4.6', '-', '-', '-']

In [349]:
Pros

['-', '-', '-', '-', '-', '-', '-']

In [350]:
Cons

['-', '-', '-', '-', '-', '-', '-']

In [361]:
#printing lengths
print(len(Laptop_name),len(Seller),len(PRICE),len(OS_),len(Display),len(Processor),len(Memory),len(Weight),len(Graphics),len(Description),len(Market_status),len(Release_date),len(Official_website),len(Display_rating),len(CPU_rating),len(GPU_rating),len(RAM_rating),len(Storage_rating),len(Overall_rating),len(Availibility),len(Digit_rating),len(User_rating),len(Pros),len(Cons),len(Laptop_URL))


7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7


In [363]:

#creating a dataframe

Top10_Laptops=pd.DataFrame({})
Top10_Laptops['Laptop_name']=Laptop_name
Top10_Laptops['Seller']=Seller
Top10_Laptops['Price']=PRICE
Top10_Laptops['Operating_system']=OS_
Top10_Laptops['Display']=Display
Top10_Laptops['Processor']=Processor
Top10_Laptops['Memory']=Memory
Top10_Laptops['Weight']=Weight
Top10_Laptops['Graphics']=Graphics
Top10_Laptops['Description']=Description
Top10_Laptops['Market_status']=Market_status
Top10_Laptops['Release_date']=Release_date
Top10_Laptops['Official_website']=Official_website
Top10_Laptops['Display_rating']=Display_rating
Top10_Laptops['CPU_rating']=CPU_rating
Top10_Laptops['GPU_rating']=GPU_rating
Top10_Laptops['RAM_rating']=RAM_rating
Top10_Laptops['Storage_rating']=Storage_rating
Top10_Laptops['Overall_rating']=Overall_rating
Top10_Laptops['Availibility']=Availibility
Top10_Laptops['Digit_rating']=Digit_rating
Top10_Laptops['User_rating']=User_rating
Top10_Laptops['Pros']=Pros
Top10_Laptops['Cons']=Cons
Top10_Laptops['Laptop_URL']=Laptop_URL

In [364]:
Top10_Laptops

,Laptop_name,Seller,Price,Operating_system,Display,Processor,Memory,Weight,Graphics,Description,...,GPU_rating,RAM_rating,Storage_rating,Overall_rating,Availibility,Digit_rating,User_rating,Pros,Cons,Laptop_URL
0,ALIENWARE AREA 51M R2,N/A,N/A,OS WINDOWS 10 HOME,"17.3"" (1920 X 1080)",10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ,1 TB SSD/16 GBGB DDR4,4.1,,If you want the absolute best when it comes to...,...,-,-,-,-,-,-,-,-,-,https://www.digit.in/laptops/alienware-area-51...
1,ALIENWARE M15 R3,N/A,₹341990,OS WINDOWS 10 HOME,"15.6"" (3840 X 2160)",10TH GENERATION INTEL® CORE™ I9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,When the ASUS ROG Strix Scar 15 ended up on ou...,If the Area 51M is a bit too much for you then...,...,-,-,-,-,-,-,-,-,-,https://www.digit.in/laptops/alienware-m15-r3-...
2,ASUS ROG STRIX SCAR 15,N/A,N/A,OS WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN™ 9 5900HX | 3.3 GHZ,1 TB SSD/16 GBGB DDR4,2.30,NVIDIA® GeForce RTX™ 3070,When the ASUS ROG Strix Scar 15 ended up on ou...,...,-,-,-,-,-,73,-,-,-,https://www.digit.in/laptops/asus-rog-strix-sc...
3,ASUS ROG ZEPHYRUS G14,amazon,₹164990,OS WINDOWS 10 HOME,"14"" (1920 X 1080)",AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1 TB SSD/16 GBGB DDR4,1.65,NVIDIA GeForce RTX 2060,The Asus Zephyrus G14 is a first-of-its-kind g...,...,-,-,-,-,Available,71,4.6,-,-,https://www.digit.in/laptops/asus-rog-zephyrus...
4,LENOVO LEGION 5I,amazon,₹76988,OS WINDOWS 10 PRO,"15.6"" (1920 X 1080)",Graphics Processor,1 TB SSD/16 GBGB DDR4,2.3,NVIDIA® GeForce® GTX 1650 4GB,Lenovo's Legion 5i is also a suitable laptop f...,...,-,-,-,-,Available,-,-,-,-,https://www.digit.in/laptops/lenovo-legion-5i-...
5,ASUS ROG ZEPHYRUS DUO 15,amazon,₹185000,OS WINDOWS 10,"15.6"" (3840 X 1100)",Processor,512 GB SSD/4 GBGB DDR4,2.4,NVIDIA GeForce RTX 2070 Max-Q,The machine is powered by an Intel Core i7-108...,...,-,-,-,-,Available,-,-,-,-,https://www.digit.in/laptops/asus-rog-zephyrus...
6,ACER ASPIRE 7 GAMING,amazon,₹79400,OS WINDOWS 10 HOME,Display,INTEL CORE I7 10TH GEN 10875H | NA,512 GB SSD/8 GBGB DDR4,2.15,NVIDIA® GeForce® GTX 1650,The Acer Aspire 7 gaming laptop is a very capa...,...,-,-,-,-,Available,75,-,-,-,https://www.digit.in/laptops/acer-aspire-7-gam...


In [365]:
#Saving the File
Top10_Laptops.to_csv("Gaming laptops_digit.csv") 

8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”,
“Industry”.

In [515]:
#searching for billionaries
url='https://www.forbes.com/real-time-billionaires/#51984ba73d78'
#getting url
driver.get(url)

In [516]:
#Extracting urls.
urls=[]
try:
    url=driver.find_elements_by_xpath('//a[@class="ng-scope"]')
    for i in url:
        urls.append(i.get_attribute('href'))
except NoSuchElementException:
    urls.append('-')
urls

['https://www.forbes.com/profile/jeff-bezos/?list=rtb/',
 'https://www.forbes.com/profile/bernard-arnault/?list=rtb/',
 'https://www.forbes.com/profile/elon-musk/?list=rtb/',
 'https://www.forbes.com/profile/mark-zuckerberg/?list=rtb/',
 'https://www.forbes.com/profile/bill-gates/?list=rtb/',
 'https://www.forbes.com/profile/larry-page/?list=rtb/',
 'https://www.forbes.com/profile/larry-ellison/?list=rtb/',
 'https://www.forbes.com/profile/sergey-brin/?list=rtb/',
 'https://www.forbes.com/profile/warren-buffett/?list=rtb/',
 'https://www.forbes.com/profile/francoise-bettencourt-meyers/?list=rtb/',
 'https://www.forbes.com/profile/steve-ballmer/?list=rtb/',
 'https://www.forbes.com/profile/mukesh-ambani/?list=rtb/',
 'https://www.forbes.com/profile/amancio-ortega/?list=rtb/',
 'https://www.forbes.com/profile/zhong-shanshan/?list=rtb/',
 'https://www.forbes.com/profile/carlos-slim-helu/?list=rtb/',
 'https://www.forbes.com/profile/jim-walton/?list=rtb/',
 'https://www.forbes.com/profile/

In [517]:
#Extracting Rank data.
Rank=[]
for i in urls:
    driver.get(i)
    try:
        rank=driver.find_elements_by_xpath('//span[@class="profile-heading__rank"]')
        for i in rank:
            Rank.append(i.text)
    except NoSuchElementException:
        Rank.append('-')
Rank


['#1',
 '#2',
 '#3',
 '#4',
 '#5',
 '#6',
 '#7',
 '#8',
 '#9',
 '#10',
 '#11',
 '#12',
 '#13',
 '#14',
 '#15',
 '#16',
 '#17',
 '#18',
 '#19',
 '#20',
 '#21',
 '#22',
 '#23',
 '#24',
 '#25']

In [518]:
#Extracting name data.
name=[]
for i in urls:
    driver.get(i)
    try:
        billionaire=driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[1]/div[3]/h1')
        for i in billionaire:
            name.append(i.text)
    except NoSuchElementException:
        name.append('-')
name

['#1 Jeff Bezos',
 '#2 Bernard Arnault & family',
 '#3 Elon Musk',
 '#4 Mark Zuckerberg',
 '#5 Bill Gates',
 '#6 Larry Page',
 '#7 Larry Ellison',
 '#8 Sergey Brin',
 '#9 Warren Buffett',
 '#10 Francoise Bettencourt Meyers & family',
 '#11 Steve Ballmer',
 '#12 Mukesh Ambani',
 '#13 Amancio Ortega',
 '#14 Zhong Shanshan',
 '#15 Carlos Slim Helu & family',
 '#16 Jim Walton',
 '#17 Alice Walton',
 '#18 Rob Walton',
 '#19 MacKenzie Scott',
 '#20 Phil Knight & family',
 '#21 Michael Bloomberg',
 '#22 François Pinault & family',
 '#23 Gautam Adani & family',
 '#24 Ma Huateng',
 '#25 Michael Dell']

In [519]:
#Extracting net worth data.
net_worth=[]
for i in urls:
    driver.get(i)
    try:
        worth=driver.find_elements_by_xpath('//div[@class="profile-info__item-value"]')
        for i in worth:
            net_worth.append(i.text)
    except NoSuchElementException:
        net_worth.append('-')
net_worth

['$209.2B',
 '$190.9B',
 '$160.1B',
 '$133B',
 '$131.2B',
 '$115.6B',
 '$112.6B',
 '$111.9B',
 '$101.1B',
 '$89.8B',
 '$85B',
 '$80.1B',
 '$78.5B',
 '$71.4B',
 '$70.6B',
 '$65B',
 '$64.7B',
 '$64.2B',
 '$61.5B',
 '$60.1B',
 '$59B',
 '$55.7B',
 '$54.6B',
 '$52.2B',
 '$49.5B']

In [520]:
#Extracting age data
Age=[]
for i in urls:
    driver.get(i)
    try:
        age=driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[1]/div[8]/div[2]/span[2]')
        for i in age:
            Age.append(i.text)
    except NoSuchElementException:
        Age.append('-')
Age

['57',
 '72',
 '50',
 '37',
 '65',
 '48',
 '76',
 '47',
 '90',
 '68',
 '65',
 '64',
 '85',
 '66',
 '81',
 '73',
 '71',
 '76',
 '51',
 '83',
 '79',
 '84',
 '59',
 '49',
 '56']

In [521]:
#Extracting citizenship.
citizenship=[]
for i in urls:
    driver.get(i)
    try:
        citizen=driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[1]/div[8]/div[5]/span[2]')
        for i in citizen:
            citizenship.append(i.text)
    except NoSuchElementException:
        citizenship.append('-')
citizenship

['Seattle, Washington',
 'France',
 'Austin, Texas',
 'Palo Alto, California',
 'Medina, Washington',
 'Palo Alto, California',
 'Lanai, Hawaii',
 'Los Altos, California',
 'Omaha, Nebraska',
 'France',
 'Hunts Point, Washington',
 'India',
 'Spain',
 'China',
 'Mexico',
 'Bentonville, Arkansas',
 'Fort Worth, Texas',
 'Bentonville, Arkansas',
 'Seattle, Washington',
 'Hillsboro, Oregon',
 'New York, New York',
 'France',
 'India',
 'China',
 'Austin, Texas']

In [522]:
# Extracting source data.
source=[]
for i in urls:
    driver.get(i)
    try:
        area=driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[1]/div[8]/div[3]/span[2]')
        for i in area:
            source.append(i.text)
    except NoSuchElementException:
        source.append('-')
source

['Amazon, Self Made',
 'LVMH',
 'Tesla, SpaceX, Self Made',
 'Facebook, Self Made',
 'Microsoft, Self Made',
 'Google, Self Made',
 'software, Self Made',
 'Google, Self Made',
 'Berkshire Hathaway, Self Made',
 "L'Oréal",
 'Microsoft, Self Made',
 'diversified',
 'Zara, Self Made',
 'beverages, pharmaceuticals, Self Made',
 'telecom, Self Made',
 'Walmart',
 'Walmart',
 'Walmart',
 'Amazon',
 'Nike, Self Made',
 'Bloomberg LP, Self Made',
 'luxury goods, Self Made',
 'infrastructure, commodities, Self Made',
 'internet media, Self Made',
 'Dell computers, Self Made']

In [523]:
#Extracting industry data.
industry=[]
for i in urls:
    driver.get(i)
    try:
        field=driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[2]/div[2]/fbs-carousel/div[1]/div/div[2]/div/div[2]/a/div')
        for i in field:
            industry.append(i.text)
    except NoSuchElementException:
        industry.append('-')
industry

['Amazon',
 'Hermès International',
 'Tesla',
 'Facebook',
 'Microsoft',
 'Alphabet',
 'Oracle',
 'Alphabet',
 'Berkshire Hathaway',
 'France',
 'Microsoft',
 'Reliance Industries',
 'Inditex',
 'Grupo Carso',
 'Walmart',
 'Walmart',
 'Walmart',
 'Amazon',
 'Nike',
 'Charles Zegar',
 'Kering',
 'Adani Ports & Special Economic Zone',
 'Tencent Holdings',
 'Dell Technologies']

9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [471]:
#searching for youtube
url='https://www.youtube.com/'
#getting url
driver.get(url)

In [472]:
#Sending keys as web scraping.
title=driver.find_element_by_xpath('//*[@id="search"]')
title.send_keys('web scraping')

In [473]:
#Clicking search button.
search=driver.find_element_by_xpath('//*[@id="search-icon-legacy"]/yt-icon')
search.click()

In [474]:
#searching for youtube
url='https://www.youtube.com/watch?v=xfzGZB4HhEE&list=RDCMUC8butISFwT-Wl7EV0hUK0BQ&index=5'
#getting url
driver.get(url)

In [475]:
#Getting web elements.
commenturl=driver.find_element_by_xpath('//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')
commenturl

<selenium.webdriver.remote.webelement.WebElement (session="dfec9b221d4283dfd7be499077838605", element="bf954456-a3f1-468f-a620-16318f28f10d")>

In [509]:
# Extracting public commrnts data.
publiccomments=[]
comments=driver.find_elements_by_xpath('//*[@id="content-text"]')
for i in comments[:500]:
    publiccomments.append(i.text)
publiccomments

['freakin hell! how good is this! Lockdown boredom solved. Thanks!',
 "Sir, you clearly explained both financial and coding keywords better than my professors did in 4 years of schooling. You're a great teacher!!",
 "Note that there are a few pops in Nick's sound at the beginning, but these go away completely once he starts building projects at the 17:20 mark.",
 'I like how the errors were made without editing them out from the clip. Thanks for including all those in.',
 'Thank you for this video but I think everybody is feeling the absence of the second section. Any updates would be greatly appreciated.',
 'I can appreciate very much the way the whole process is explained, however looping through Pandas DataFrame for simple math calculations is not "the Pandas way" of doing things :)',
 "I  invested in Tesla back in 2013, I was very much a bear. My reasoning was simple: Tesla was trying to do what no American automaker had done so far: Develop and sell a mass-market electric vehicle.

In [510]:
#Seeing lenght of public comments data.
len(publiccomments)

500

In [511]:
# Extracting time data.
time=[]
commenttime=driver.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a')
for i in commenttime[:500]:
    time.append(i.text)
time

['7 months ago',
 '7 months ago',
 '7 months ago',
 '7 months ago (edited)',
 '6 months ago',
 '1 week ago',
 '5 hours ago',
 '7 months ago',
 '7 months ago',
 '7 months ago',
 '4 months ago',
 '1 hour ago',
 '5 months ago',
 '4 months ago',
 '6 months ago',
 '7 months ago',
 '4 months ago',
 '7 months ago',
 '7 months ago (edited)',
 '7 months ago',
 '5 months ago',
 '6 months ago',
 '3 months ago',
 '7 months ago',
 '1 month ago',
 '5 months ago',
 '5 months ago',
 '4 months ago',
 '7 months ago',
 '7 months ago',
 '2 months ago',
 '7 months ago',
 '4 months ago',
 '7 months ago',
 '7 months ago',
 '4 weeks ago (edited)',
 '4 months ago',
 '1 week ago',
 '7 months ago',
 '6 months ago',
 '6 months ago',
 '3 months ago',
 '7 months ago',
 '3 weeks ago',
 '7 months ago',
 '3 weeks ago',
 '4 months ago',
 '5 months ago (edited)',
 '5 months ago',
 '7 months ago',
 '7 months ago',
 '5 months ago',
 '7 months ago',
 '5 months ago',
 '7 months ago',
 '6 months ago',
 '3 months ago',
 '4 mo

In [512]:
#Seeing lenght of time data.
len(time)

500

In [513]:
#Extracting vote data.
vote=[]
try:
    upvote=driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
    for i in upvote[:500]:
        vote.append(i.text)
except NoSuchElementException:
    vote.append('-')
vote

['5',
 '457',
 '201',
 '100',
 '37',
 '11',
 '6',
 '2',
 '55',
 '8',
 '29',
 '',
 '11',
 '1',
 '',
 '41',
 '1',
 '51',
 '39',
 '11',
 '1',
 '1',
 '',
 '28',
 '3',
 '1',
 '',
 '1',
 '2',
 '31',
 '1',
 '2',
 '11',
 '1',
 '30',
 '',
 '',
 '1',
 '',
 '10',
 '1',
 '',
 '1',
 '',
 '8',
 '',
 '',
 '',
 '',
 '41',
 '',
 '',
 '',
 '',
 '27',
 '1',
 '1',
 '1',
 '1',
 '32',
 '',
 '1',
 '',
 '',
 '5',
 '',
 '',
 '',
 '',
 '31',
 '1',
 '',
 '',
 '1',
 '6',
 '',
 '',
 '',
 '',
 '3',
 '',
 '',
 '2',
 '',
 '10',
 '1',
 '',
 '1',
 '',
 '3',
 '',
 '',
 '',
 '',
 '6',
 '',
 '1',
 '',
 '',
 '6',
 '',
 '',
 '',
 '',
 '13',
 '',
 '',
 '4',
 '',
 '3',
 '',
 '',
 '',
 '',
 '31',
 '',
 '',
 '',
 '',
 '4',
 '',
 '4',
 '',
 '8',
 '480',
 '1',
 '',
 '3',
 '',
 '10',
 '',
 '1',
 '',
 '',
 '4',
 '2',
 '1',
 '4',
 '',
 '14',
 '1',
 '',
 '',
 '',
 '3',
 '',
 '',
 '2',
 '',
 '41',
 '',
 '',
 '',
 '',
 '35',
 '',
 '',
 '',
 '',
 '6',
 '',
 '2',
 '',
 '',
 '9',
 '',
 '1',
 '',
 '',
 '15',
 '1',
 '',
 '1',
 '',
 '22',
 '

In [514]:
#Lenght of vote data.
len(vote)

500

10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [413]:
#connecting to the webdriver
driver = webdriver.Chrome(r"/Users/prajwalkunder/Downloads/chromedriver")

In [447]:
#searching for hostles in london
url='https://www.hostelworld.com/ '
#getting url
driver.get(url)

In [448]:
#clicking search bitton
title=driver.find_element_by_xpath('//*[@id="location-text-input-field"]')
title.click()

In [449]:
#sending keys as london
search=driver.find_element_by_xpath('//*[@id="location-text-input-field"]')
search.send_keys('London')

In [450]:
#clicking calendar
calendar=driver.find_element_by_xpath('//i[@class="core-icon icon-core-calendar"]')
calendar.click()

In [451]:
#clicking check out
check_out=driver.find_element_by_xpath('//div[@class="field-inner"]/span')
check_out.click()

In [452]:
#clicking guests
guests=driver.find_element_by_xpath('//span[@class="icon"]')
guests.click()

In [453]:
#clicking title
title=driver.find_element_by_xpath('//input[@class="location-text"]')
title.click()

In [454]:
#sending keys as London
search=driver.find_element_by_xpath('//*[@id="location-text-input-field"]')
search.send_keys('London')

In [455]:
search=driver.find_element_by_xpath('//*[@id="search-button"]')
search.click()

In [456]:
#Extracting hostel name data
hostel_name=[]
try:
    name=driver.find_elements_by_xpath('//h2[@class="title title-6"]/a')
    for i in name:
        hostel_name.append(i.text)
except NoSuchElementException:
    hostel_name.append('-')
hostel_name

['Generator London',
 'Safestay London Kensington Holland Park',
 'PubLove @ The Crown, Battersea',
 '247london Hostel and Private Rooms',
 'PubLove @ The Steam Engine, Waterloo',
 'The London Home Hostel',
 "Wombat's The City Hostel London",
 'Safestay London Elephant & Castle',
 'PubLove @ The Green Man, Paddington',
 'Astor Museum Inn',
 'PubLove @ The White Ferry, Victoria',
 'No.8 Willesden Hostel London',
 'YHA London St Pancras',
 'SoHostel',
 'The Walrus Hostel',
 'Keystone House Kings Cross',
 'Clink261',
 'Burlington Residence',
 'London Waterloo Hostel',
 'Jaeger Acton Hostel',
 'Astor Hyde Park',
 'Abercorn House',
 'Astor Kensington',
 'New Cross Inn Hostel',
 'Mapesbury Hostel',
 'Hootananny Hostel',
 'Phoenix Hostel',
 'Smart Hyde Park Inn Hostel',
 'Well Street Hall',
 'YHA London Central']

In [457]:
#Extracting distance data.
distance=[]
kms=driver.find_elements_by_xpath('//span[@class="description"]')
for i in kms:
    distance.append(i.text)
distance

['Hostel - 3km from city centre',
 'Hostel - 5.9km from city centre',
 'Hostel - 4.7km from city centre',
 'Hostel - 8.2km from city centre',
 'Hostel - 0.5km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 3.6km from city centre',
 'Hostel - 1.7km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 2.2km from city centre',
 'Hostel - 2.4km from city centre',
 'Hostel - 10km from city centre',
 'Hostel - 3.3km from city centre',
 'Hostel - 2km from city centre',
 'Hostel - 0.2km from city centre',
 'Hostel - 3.5km from city centre',
 'Hostel - 3.2km from city centre',
 'Hostel - 7.1km from city centre',
 'Hostel - 0.7km from city centre',
 'Hostel - 10.4km from city centre',
 'Hostel - 4.3km from city centre',
 'Hostel - 7.3km from city centre',
 'Hostel - 4.9km from city centre',
 'Hostel - 6.1km from city centre',
 'Hostel - 8.2km from city centre',
 'Hostel - 5km from city centre',
 'Hostel - 4.2km from city centre',
 'Hostel - 5km from city centre',


In [458]:
#Extracting reviews data.
reviews=[]
review=driver.find_elements_by_xpath('//div[@class="keyword"]')
for i in review:
    reviews.append(i.text)
reviews

['Superb',
 'Superb',
 'Fabulous',
 'Very Good',
 'Superb',
 'Good',
 'No Rating',
 'Fabulous',
 'Very Good',
 'Superb',
 'Fabulous',
 'Good',
 'Superb',
 'Fabulous',
 'Fabulous',
 'Superb',
 'Fabulous',
 'Fabulous',
 'Superb',
 'Fabulous',
 'Superb',
 'Fabulous',
 'Superb',
 'Superb',
 'Very Good',
 'Fabulous',
 'Very Good',
 'Rating',
 'Fabulous',
 'Fabulous',
 'Very Good',
 'NEW',
 'Very Good']

In [459]:
#Extracting rating data.
rating=[]
hostlerating=driver.find_elements_by_xpath('//div[@class="score orange big"]')
for i in hostlerating:
    rating.append(i.text)
rating

['7.6',
 '9.8',
 '6.8',
 '8.9',
 '7.7',
 '9.3',
 '8.1',
 '6.9',
 '9.5',
 '8.9',
 '8.9',
 '9.5',
 '8.9',
 '8.6',
 '9.7',
 '8.2',
 '9.7',
 '8.0',
 '9.4',
 '9.1',
 '7.3',
 '8.9',
 '7.3',
 '8.0',
 '8.5',
 '7.6',
 '7.3']

In [460]:
#Extracting reviews data.
total_reviews=[]
reviews=driver.find_elements_by_xpath('//div[@class="reviews"]')
for i in reviews:
    total_reviews.append(i.text)  
total_reviews

['6733 Total Reviews',
 '1074 Total Reviews',
 '207 Total Reviews',
 '443 Total Reviews',
 '186 Total Reviews',
 '248 Total Reviews',
 '13130 Total Reviews',
 '3947 Total Reviews',
 '391 Total Reviews',
 '8707 Total Reviews',
 '130 Total Reviews',
 '4563 Total Reviews',
 '2775 Total Reviews',
 '3952 Total Reviews',
 '3638 Total Reviews',
 '8109 Total Reviews',
 '3167 Total Reviews',
 '26 Total Reviews',
 '2400 Total Reviews',
 '879 Total Reviews',
 '11321 Total Reviews',
 '3654 Total Reviews',
 '6324 Total Reviews',
 '999 Total Reviews',
 '357 Total Reviews',
 '1406 Total Reviews',
 '3405 Total Reviews',
 '6232 Total Reviews',
 '0 Total Reviews',
 '4693 Total Reviews']

In [461]:
#Extracting price data.
price=[]
try:
    privates=driver.find_elements_by_xpath('//div[@class="price title-5"]')
    for i in privates:
        price.append(i.text)
except NoSuchElementException:
    price.append('-')
price


['Rs8099',
 'Rs1743',
 'Rs7382',
 'Rs2214',
 'Rs1107',
 'Rs5229',
 'Rs1569',
 'Rs8202',
 'Rs9077',
 'Rs2291',
 'Rs4152',
 'Rs1015',
 'Rs7382',
 'Rs1384',
 'Rs1845',
 'Rs20761',
 'Rs2214',
 'Rs2045',
 'Rs969',
 'Rs3998',
 'Rs2255',
 'Rs1435',
 'Rs4269',
 'Rs6311',
 'Rs1532',
 'Rs2091',
 'Rs7843',
 'Rs1845',
 'Rs5126',
 'Rs4101',
 'Rs1538',
 'Rs2973',
 'Rs1743',
 'Rs1538',
 'Rs10519',
 'Rs4613',
 'Rs2435',
 'Rs3086',
 'Rs1736',
 'Rs4179',
 'Rs1164',
 'Rs4306',
 'Rs6049']

In [462]:
#Extracting dorms.
dorms=[]
try:
    cost=driver.find_elements_by_xpath('//div[@class="price title-5"]')
    for i in cost:
        dorms.append(i.text)
except NoSuchElementException:
    dorms.append('-')
dorms

['Rs8099',
 'Rs1743',
 'Rs7382',
 'Rs2214',
 'Rs1107',
 'Rs5229',
 'Rs1569',
 'Rs8202',
 'Rs9077',
 'Rs2291',
 'Rs4152',
 'Rs1015',
 'Rs7382',
 'Rs1384',
 'Rs1845',
 'Rs20761',
 'Rs2214',
 'Rs2045',
 'Rs969',
 'Rs3998',
 'Rs2255',
 'Rs1435',
 'Rs4269',
 'Rs6311',
 'Rs1532',
 'Rs2091',
 'Rs7843',
 'Rs1845',
 'Rs5126',
 'Rs4101',
 'Rs1538',
 'Rs2973',
 'Rs1743',
 'Rs1538',
 'Rs10519',
 'Rs4613',
 'Rs2435',
 'Rs3086',
 'Rs1736',
 'Rs4179',
 'Rs1164',
 'Rs4306',
 'Rs6049']

In [463]:
#Extracting urls data.
urls=[]
url=driver.find_elements_by_xpath('//div[@class="gallery"]/a')
for i in url:
    urls.append(i.get_attribute('href'))
urls


['https://www.hostelworld.com/pwa/hosteldetails.php/Wombat-s-The-City-Hostel-London/London/88047?from=2021-07-31&to=2021-08-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Selina-Camden/London/310274?from=2021-07-31&to=2021-08-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/London-Backpackers/London/527?from=2021-07-31&to=2021-08-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Generator-London/London/510?from=2021-07-31&to=2021-08-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Safestay-London-Kensington-Holland-Park/London/93920?from=2021-07-31&to=2021-08-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove-The-Crown-Battersea/London/60336?from=2021-07-31&to=2021-08-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/247london-Hostel-and-Private-Rooms/London/64523?from=2021-07-31&to=2021-08-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove-The-Steam-Engine

In [464]:
view=driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/div[4]/div/div/div[3]/div[3]/a[2]/button')
view.click()

In [465]:
#Extracting Description data.
description=[]
for i in urls:
    driver.get(i)
    try:
        hostle=driver.find_elements_by_xpath('//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div/div/div[2]')
        for i in hostle:
            description.append(i.text)
    except NoSuchElementException:
        description.append('-')
description

["Wombat's The City Hostel London is definitely the place to stay if you want to experience all that London has to offer. Only minutes away from sights such as the Tower Bridge, the Tower of London, and the City Hall, it's next to Wilton’s Music Hall and an easy walk from the buzzing nightlife of Shoreditch. The location is ideal for young travellers. Not only is the location awesome, the building itself is breath-taking. Formerly a seamen's hostel, the building was converted to an amazing travellers' oasis featuring a huge bar, a courtyard, a cosy hangout area in the lobby, and all the amenities a backpacker could wish for. * Please note: due to the pandemic, our bar remains closed at the moment.\n\nWombat's The City Hostel is proud to be one of the most experienced hostel chains (over 20 years of hostel expertise!) with a history of award-winning properties all over Europe: Budapest, London, Munich, and Vienna.\n\nWombat’s The City Hostel offers safe, clean, quality accommodation in 